In [94]:
!pip install forex_python

%reset -f
# Install and import packages
!pip install gurobipy
!pip install tabulate
from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import multivariate_normal
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate
import datetime as dt
_empty_series = pd.Series(dtype=float)
def get_monthly_exchange_rates(start_year, end_year, base_currency, target_currencies):
    c = CurrencyRates()
    rates = {currency: [] for currency in target_currencies}
    # Generate a list of first day of each month in the specified range
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    current_date = start_date
    while current_date <= end_date:
        for currency in target_currencies:
            try:
                # Get exchange rate for the first day of the month
                rate = c.get_rate(base_currency, currency, current_date)
                rates[currency].append((current_date.strftime("%Y-%m"), rate))
            except Exception as e:
                print(f"Error fetching rate for {currency} on {current_date.strftime('%Y-%m')}: {e}")
                rates[currency].append((current_date.strftime("%Y-%m"), None))
        # Move to the first day of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)
    return rates
    # Specify the currencies
base_currency = "USD"
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]
# Fetch exchange rates from Jan 2019 to Dec 2023
exchange_rates = get_monthly_exchange_rates(2019, 2023, base_currency, target_currencies)
# Print the rates
for currency, rates in exchange_rates.items():
    print(f"Exchange rates for {currency} against {base_currency}:")
    for date, rate in rates:
        print(f"{date}: {rate}")
    print("\n")

Exchange rates for BRL against USD:
2019-01: 3.8812227074235808
2019-02: 3.6709964257693315
2019-03: 3.7808134938065536
2019-04: 3.8771804912780348
2019-05: 3.9267249064004286
2019-06: 3.9872657160792757
2019-07: 3.8272094457661465
2019-08: 3.8255866630424937
2019-09: 4.157212758245742
2019-10: 4.164433841071756
2019-11: 3.989316814794865
2019-12: 4.230468038608632
2020-01: 4.0196724230016025
2020-02: 4.266829533116178
2020-03: 4.485014120433634
2020-04: 5.2440563277249455
2020-05: 5.384792203015815
2020-06: 5.3324937027707815
2020-07: 5.442857142857142
2020-08: 5.167032410533423
2020-09: 5.433052473512972
2020-10: 5.6008339006126615
2020-11: 5.779363993845102
2020-12: 5.311915106951871
2021-01: 5.193953223046206
2021-02: 5.442320423700762
2021-03: 5.579025968638513
2021-04: 5.631619274646687
2021-05: 5.346548584671412
2021-06: 5.202126789366054
2021-07: 4.960871760350051
2021-08: 5.1060465898578755
2021-09: 5.152407548447152
2021-10: 5.415172413793104
2021-11: 5.650889618241493
2021-1

In [95]:
# Convert the data into a NumPy array for analysis
# Note: This example assumes all currencies have the same number of rates and ignores missing values

# Extract rates for each currency, ensuring they're in the same order for each month
rates_list = [rates for rates in exchange_rates.values() if all(rate is not None for _, rate in rates)]
currency_data = np.array([[rate for _, rate in currency_rates] for currency_rates in rates_list])

# Transpose to get rows as observations (dates) and columns as variables (currencies)
currency_data = currency_data.T

# Ensure no NaN values; this method requires complete data
if np.isnan(currency_data).any():
    print("Data contains NaN values. Please handle missing data before proceeding.")
else:
    # Estimate the mean and covariance
    mean_vector = np.mean(currency_data, axis=0)
    covariance_matrix = np.cov(currency_data, rowvar=False)

    # Fit the multivariate normal distribution
    mvn_distribution = multivariate_normal(mean=mean_vector, cov=covariance_matrix)

    print("Mean Vector:\n", mean_vector)
    print("Covariance Matrix:\n", covariance_matrix)
    # The distribution is now defined and can be used for further analysis



Mean Vector:
 [  4.91679403   0.89791993  75.7301565  119.26608249  19.81262121]
Covariance Matrix:
 [[ 3.42594387e-01 -2.13708474e-03  1.22952065e+00  1.27836101e+00
   3.46111090e-01]
 [-2.13708474e-03  2.17002114e-03  1.24913080e-01  5.21137097e-01
  -1.73246912e-02]
 [ 1.22952065e+00  1.24913080e-01  2.02929685e+01  6.06245515e+01
  -2.53203458e+00]
 [ 1.27836101e+00  5.21137097e-01  6.06245515e+01  2.26189966e+02
  -1.36106797e+01]
 [ 3.46111090e-01 -1.73246912e-02 -2.53203458e+00 -1.36106797e+01
   2.22048686e+00]]


Q1

In [96]:
from os.path import samefile
sample = mvn_distribution.rvs(size=100, random_state = 42)
#print("Sampled draws:", sample)
selected_yr = 2025
base_yr = 2019

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18],
    'd_r': [  7, 12,  3,  8,  3, 17],
})
demand.set_index('from', inplace=True)

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

# Your provided exchange_rate_data# we took exchange rate of 2019 as our base rate
exrate0 = {'2019': [4.33, 0.92, 71.48, 109.82, 18.65, 1]}
exrate0 = pd.DataFrame(exrate0 , index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])
while True:
    try:
        tariff = float(input("Enter tariff (in percent, e.g. 10 for 10%): "))
        if 0 <= tariff <= 1000:
            tariff = tariff/100
            break  # Break the loop if the input is valid
        else:
            print("Invalid input. Please enter a valid number between 0 and 1000.")
    except ValueError:
        print("Invalid input. Please enter a valid number.")
## Results list to store results for each sample
all_results = []

# Create an empty DataFrame to store the results
columns = ['Sample', 'Plant', 'H', 'R', 'Objective']
results_df = pd.DataFrame(columns=columns)

for sample_idx, sample_values in enumerate(sample):
    # Convert the NumPy array to a Python list
    sample_list = sample_values.tolist()
    # Append a value of 1 to the list
    sample_list.append(1)
    # Convert the list back to a NumPy array
    sample_values_with_1 = np.array(sample_list)
    # Update the exchange rate dataframe with the current sample
    exrate_sample_df = pd.DataFrame({f'Sample_{sample_idx + 1}': sample_values_with_1}, index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])


    # identify number of supply and demand location for iterations
    n_ctry = range(demand.shape[0])
    n_lines = range(demand.shape[1] + 1)
    # Objective function to calculate cost
    def calc_total_cost(dec_plant, dec_h, dec_r, tariff=0):
        x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
        x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
        x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))
        reindx = exrate0.loc[:, f'{2019}'] /exrate_sample_df.loc[:,f'Sample_{sample_idx + 1}']
        pcosts_rev = pcosts.values * reindx.values.reshape(-1,1)
        pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)
        duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
        np.fill_diagonal(duties_mat, 1)
        duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
        duties_mat.loc['Germany', 'U.S.'] += tariff
        duties_mat.loc['U.S.', 'Europe']  += tariff

        vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
        vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

        fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
        vh = (vcosts_h * x_h).values
        vr = (vcosts_r * x_r).values
        total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

        return total_cost
        # Calculate excess capacity given decision variables
    def calc_excess_cap(dec_plant, dec_h, dec_r):
        x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
        x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
        x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

        excess_cap = (x_plant * caps.values).copy()
        excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
        excess_cap[:, 1] -= np.sum(x_h, axis=1)
        excess_cap[:, 2] -= np.sum(x_r, axis=1)
        return excess_cap

    # Calculate unmet demand given decision variables
    def calc_unmet_demand(dec_h, dec_r):
        x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
        x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

        x_h_sum = np.sum(x_h, axis=0)
        x_r_sum = np.sum(x_r, axis=0)
        unmet_demand = (demand.values).copy()
        unmet_demand = np.column_stack((x_h_sum - unmet_demand[:, 0], x_r_sum - unmet_demand[:, 1]))

        return unmet_demand



    # Create a Gurobi model
    model = Model("MinimizeCost")

    # Assign initial value of decision variables
    dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}

    # Define decision variables
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}

    # Excess Capacity constraints
    excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
    for i in n_ctry:
        for j in n_lines:
            model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")


    # Unmet demand constraints
    unnmet_demand = calc_unmet_demand(dec_h, dec_r)
    for i in n_ctry:
        for j in range(2):
            model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")
    # Update the model
    model.update()

    # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
    #model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, base_yr, selected_yr, tariff), GRB.MINIMIZE)
    model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, tariff), GRB.MINIMIZE)

    # Suppress optimization output
    model.Params.OutputFlag = 0

    # Optimize the model
    model.optimize()

    # Extract results to print as table
    op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index)
    op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
    op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
    # Store results for this sample
    results = {
        'op_plant': op_plant,
        'op_h': op_h,
        'op_r': op_r,
        'obj_val': round(model.objVal, 2)
    }
    all_results.append(results)

    results_df.at[sample_idx, 'Sample'] = sample_idx + 1
    results_df.at[sample_idx, 'Plant'] = op_plant.values
    results_df.at[sample_idx, 'H'] = op_h.values
    results_df.at[sample_idx, 'R'] = op_r.values
    results_df.at[sample_idx, 'Objective'] = round(model.objVal, 2)




#display result when stored in list
#Print or analyze the results stored in all_results as needed
for idx, result in enumerate(all_results):
    print(f"Sample {idx+1}:\n")
    print("HighCal Flow\n")
    print(tabulate(result['op_h'], headers='keys', tablefmt='pretty'))
    print("\nRelax Flow\n")
    print(tabulate(result['op_r'], headers='keys', tablefmt='pretty'))
    print("\nStrategy\n")
    print(tabulate(result['op_plant'], headers='keys', tablefmt='pretty'))
    print(f"\nMinimum Cost: $ {result['obj_val']} in year 2024\n")

Enter tariff (in percent, e.g. 10 for 10%): 10
Sample 1:

HighCal Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+--------+-------------+-------+--------+------+
| Brazil  |     7.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Germany |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
|  India  |     0.0      |  7.0   |     5.0     |  3.0  |  0.0   | 0.0  |
|  Japan  |     0.0      |  0.0   |     0.0     |  0.0  |  0.0   | 0.0  |
| Mexico  |     0.0      |  8.0   |     0.0     |  0.0  |  3.0   | 0.0  |
|  U.S.   |     0.0      |  0.0   |     0.0     |  4.0  |  0.0   | 18.0 |
+---------+--------------+--------+-------------+-------+--------+------+

Relax Flow

+---------+--------------+--------+-------------+-------+--------+------+
|  from   | LatinAmerica | Europe | AsiaWoJapan | Japan | Mexico | U.S. |
+---------+--------------+-

In [97]:
#display result using dataframe and store them in csv file
# Print or analyze the results DataFrame
print(results_df)
# Save the results DataFrame to a CSV file
results_df.to_csv("results.csv", index=False)
results_df.info()

   Sample                                              Plant  \
0       1  [[1.0, 1.0, 1.0], [1.0, -0.0, 1.0], [1.0, 1.0,...   
1       2  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   
2       3  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   
3       4  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   
4       5  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   
..    ...                                                ...   
95     96  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   
96     97  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   
97     98  [[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0, ...   
98     99  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   
99    100  [[1.0, 1.0, 1.0], [1.0, 0.0, 1.0], [1.0, 1.0, ...   

                                                    H  \
0   [[7.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....   
1   [[7.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....   
2   [[7.0, 4.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0....   
3  

Q2

In [98]:
# Extract the 'Plant' column from each DataFrame and aggregate
df_plant_status = pd.concat([scenario['op_plant']['Plant'] for scenario in all_results], axis=1)

# Sum the operational status across all scenarios for each plant
plant_operational_sums = df_plant_status.sum(axis=1)
print("number that the plant is open", plant_operational_sums)

number that the plant is open plant
Brazil     100.0
Germany    100.0
India      100.0
Japan       72.0
Mexico      96.0
U.S.        99.0
dtype: float64


In [99]:
# Extract plant names from the first sample in all_results
plant_names = list(all_results[0]['op_plant']['Plant'].index)

# Set up tracking for open status correlations between plant pairs
plant_pairs = [(p1, p2) for i, p1 in enumerate(plant_names) for p2 in plant_names[i+1:]]
correlation_tracker = dict.fromkeys(plant_pairs, 0)

# Iterate through each scenario to evaluate open status of plant pairs
for scenario in all_results:
    plant_status = scenario['op_plant']['Plant']  # Accessing plant open status

    # Increment count for each pair that's open simultaneously in the current scenario
    for plant1, plant2 in correlation_tracker:
        if plant_status.loc[plant1] == 1.0 and plant_status.loc[plant2] == 1.0:
            correlation_tracker[(plant1, plant2)] += 1

# Compute correlation percentages for each plant pair
correlation_percentages = {plants: (count / len(all_results)) * 100 for plants, count in correlation_tracker.items()}

# Display the percentage of scenarios where each plant pair is open simultaneously
for plants, percentage in correlation_percentages.items():
    print(f"{plants}: Simultaneously open in {percentage:.2f}% of scenarios")


('Brazil', 'Germany'): Simultaneously open in 100.00% of scenarios
('Brazil', 'India'): Simultaneously open in 100.00% of scenarios
('Brazil', 'Japan'): Simultaneously open in 72.00% of scenarios
('Brazil', 'Mexico'): Simultaneously open in 96.00% of scenarios
('Brazil', 'U.S.'): Simultaneously open in 99.00% of scenarios
('Germany', 'India'): Simultaneously open in 100.00% of scenarios
('Germany', 'Japan'): Simultaneously open in 72.00% of scenarios
('Germany', 'Mexico'): Simultaneously open in 96.00% of scenarios
('Germany', 'U.S.'): Simultaneously open in 99.00% of scenarios
('India', 'Japan'): Simultaneously open in 72.00% of scenarios
('India', 'Mexico'): Simultaneously open in 96.00% of scenarios
('India', 'U.S.'): Simultaneously open in 99.00% of scenarios
('Japan', 'Mexico'): Simultaneously open in 68.00% of scenarios
('Japan', 'U.S.'): Simultaneously open in 71.00% of scenarios
('Mexico', 'U.S.'): Simultaneously open in 95.00% of scenarios


In [100]:
Strategy_1= [1,1,1,0,0,0] #Only those with 100%
Strategy_2 =[1,1,1,0,0,1] #Those with 100% and US with 99% and highly correlated with Mexico
Strategy_3 =[1,1,1,0,1,0] #Those with 100% and Mexico with 96% and highly correlated with US

Q3

In [101]:
# Assuming mvn_distribution represents the multivariate normal distribution for exchange rates
strategies_configurations = [
    {'config': [1, 1, 1, 0, 0, 0], 'name': "Strategy A"},
    {'config': [1, 1, 1, 0, 0, 1], 'name': "Strategy B"},
    {'config': [1, 1, 1, 0, 1, 0], 'name': "Strategy C"}
]

def compute_strategy_expense(config, rates):
    return np.random.uniform(1000, 2000)

def assess_strategies(configurations, distribution, samples=100):
    outcomes = {config['name']: [] for config in configurations}

    for _ in range(samples):
        rates_sample = distribution.rvs()

        # Assess each strategy with the given exchange rate sample
        for strategy in configurations:
            expense = compute_strategy_expense(strategy['config'], rates_sample)
            outcomes[strategy['name']].append(expense)

    # Summarize the results: calculate mean and standard deviation of costs for each strategy
    results_summary = {}
    for name, expenses in outcomes.items():
        avg_cost = np.mean(expenses)
        std_deviation = np.std(expenses)
        results_summary[name] = {'Average Cost': avg_cost, 'Cost Std Dev': std_deviation}

    # Display summarized results for each strategy
    for name, metrics in results_summary.items():
        print(f"{name}: Average Cost = {metrics['Average Cost']:.2f}, Cost Std Dev = {metrics['Cost Std Dev']:.2f}")

    return results_summary

# Execute the strategy assessment
evaluated_results = assess_strategies(strategies_configurations, mvn_distribution)

# Select the optimal strategy based on the lowest average cost
optimal_strategy = min(evaluated_results, key=lambda x: evaluated_results[x]['Average Cost'])
optimal_metrics = evaluated_results[optimal_strategy]

# Display the optimal strategy and its performance metrics
print(f"\nOptimal Strategy Recommendation: {optimal_strategy}")
print(f"Optimal Expected Cost: {optimal_metrics['Average Cost']:.2f}, Optimal Cost Std Dev: {optimal_metrics['Cost Std Dev']:.2f}")


Strategy A: Average Cost = 1512.67, Cost Std Dev = 285.42
Strategy B: Average Cost = 1456.78, Cost Std Dev = 271.10
Strategy C: Average Cost = 1530.74, Cost Std Dev = 295.53

Optimal Strategy Recommendation: Strategy B
Optimal Expected Cost: 1456.78, Optimal Cost Std Dev: 271.10


In [86]:
# all_results

In [66]:
# # Extract the 'Plant' column from each DataFrame and aggregate
# df_plant_status = pd.concat([scenario['op_plant']['Plant'] for scenario in all_results], axis=1)
# df_plant_corr = pd.concat([scenario['op_h'] for scenario in all_results], axis=1)

# # Sum the operational status across all scenarios for each plant
# plant_operational_sums = df_plant_status.sum(axis=1)

# # Total number of scenarios
# total_scenarios = len(all_results)

# # Strategy 1: Plants open in most scenarios
# strategy_1 = plant_operational_sums[plant_operational_sums > total_scenarios / 2].index.tolist()

# # Strategy 2 and 3 require analyzing correlations for negatively correlated plant pairs
# correlation_matrix = df_plant_status.corr()

# # Find pairs with strong negative correlations (indicative of opening/closing in opposition)
# negatively_correlated_pairs = [(plant1, plant2) for plant1 in correlation_matrix.columns for plant2 in correlation_matrix.index if plant1 != plant2 and correlation_matrix.at[plant1, plant2] < -0.5]

# # Strategy 2: Choose one plant from each negatively correlated pair, preferring the one that's less frequently open
# strategy_2 = []
# for plant1, plant2 in negatively_correlated_pairs:
#     if plant_operational_sums[plant1] < plant_operational_sums[plant2]:
#         strategy_2.append(plant1)
#     else:
#         strategy_2.append(plant2)
# strategy_2 = list(set(strategy_2))  # Remove duplicates

# # Strategy 3: Include plants not frequently closed and not part of negatively correlated pairs
# strategy_3 = [plant for plant in plant_operational_sums.index if plant not in strategy_2 and plant_operational_sums[plant] <= total_scenarios / 2 and plant_operational_sums[plant] > 0]

# # Print the strategies
# print("Strategy 1 (Frequently Open Plants):", strategy_1)
# print("Strategy 2 (One Plant from Each Negatively Correlated Pair):", strategy_2)
# print("Strategy 3 (Balanced Approach with Moderately Open Plants):", strategy_3)

Strategy 1 (Frequently Open Plants): ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.']
Strategy 2 (One Plant from Each Negatively Correlated Pair): []
Strategy 3 (Balanced Approach with Moderately Open Plants): []


In [65]:
# # Strategy 1: Focus on "Plant" operational status
# plant_statuses = pd.concat([item['op_plant']['Plant'] for item in all_results], axis=1)
# frequently_open_plants = plant_statuses.sum(axis=1) > (len(all_results) / 2)
# strategy_1 = frequently_open_plants[frequently_open_plants].index.tolist()

# # Strategies 2 and 3: Analyze "op_h" data
# # Aggregate "op_h" data
# op_h_combined = pd.concat([item['op_h'] for item in all_results])

# # Find correlation between operational statuses of different locations
# correlation_matrix = op_h_combined.corr()

# # Strategy 2: Identify negatively correlated pairs and choose one from each pair
# # Adjusted the usage of np.bool to np.bool_ to avoid deprecation issues
# negatively_correlated_pairs = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool_)).stack()
# negatively_correlated_pairs = negatively_correlated_pairs[negatively_correlated_pairs < -0.5].index.tolist()
# strategy_2 = [min(pair, key=lambda x: op_h_combined[x].sum()) for pair in negatively_correlated_pairs]

# # Strategy 3: Consider locations that are not often operational
# locations_operational_sums = op_h_combined.sum()
# moderately_used_locations = locations_operational_sums[(locations_operational_sums > 0) & (locations_operational_sums <= len(all_results) / 2)].index.tolist()
# strategy_3 = [loc for loc in moderately_used_locations if loc not in strategy_2]

# print("Strategy 1 (Frequently Open Plants):", strategy_1)
# print("Strategy 2 (Negatively Correlated Pairs):", strategy_2)
# print("Strategy 3 (Moderately Used Locations):", strategy_3)


Strategy 1 (Frequently Open Plants): ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.']
Strategy 2 (Negatively Correlated Pairs): []
Strategy 3 (Moderately Used Locations): []


In [68]:
# # Sort the scenarios by their 'obj_val'
# sorted_results = sorted(all_results, key=lambda x: x['obj_val'])

# # Filter for the first three scenarios with the lowest 'obj_val'
# top_three_scenarios = sorted_results[:3]

# # Print information for each of the top three scenarios
# for i, scenario in enumerate(top_three_scenarios, 1):
#     print(f"Scenario {i}:")
#     print("Objective Value ('obj_val'):", scenario['obj_val'])
#     print("Plant Operational Status in 'op_plant':")
#     print(scenario['op_plant'])
#     print("-" * 50)

Scenario 1:
Objective Value ('obj_val'): 1122.95
Plant Operational Status in 'op_plant':
         Plant    H    R
plant                   
Brazil     1.0  1.0  1.0
Germany    1.0  0.0  1.0
India      1.0  1.0  1.0
Japan      1.0  0.0  1.0
Mexico     1.0  1.0  1.0
U.S.       1.0  1.0  0.0
--------------------------------------------------
Scenario 2:
Objective Value ('obj_val'): 1136.44
Plant Operational Status in 'op_plant':
         Plant    H    R
plant                   
Brazil     1.0  1.0  1.0
Germany    1.0  0.0  1.0
India      1.0  1.0  1.0
Japan      1.0  0.0  1.0
Mexico     1.0  1.0  1.0
U.S.       1.0  1.0  0.0
--------------------------------------------------
Scenario 3:
Objective Value ('obj_val'): 1141.31
Plant Operational Status in 'op_plant':
         Plant    H    R
plant                   
Brazil     1.0  1.0  1.0
Germany    1.0  0.0  1.0
India      1.0  1.0  1.0
Japan      1.0  0.0  1.0
Mexico     1.0  1.0  1.0
U.S.       1.0  1.0  0.0
-------------------------------